## NOTE: If you are running this notebook on Google Colab,
## then uncomment the two cells below and then run them

In [ ]:
# !pip install datasets evaluate --upgrade
# !python -m spacy download en_core_web_sm

In [ ]:
# pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 torchtext==0.14.1
# pip install datasets evaluate

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [ ]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## Import Dataset that i have preprocessed and uploaded in hugging face

In [13]:
from datasets import load_dataset
dataset = load_dataset("apurbapokharel/deliconv")

#### Dataset is split into train, test and validation

In [14]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

# train_data, test_data = (
#     dataset["train"],
#     dataset["test"]
# )
# train_data = train_data.select(range(10000))
# test_data = test_data.select(range(10000))

In [16]:
train_data

Dataset({
    features: ['Key', 'Data'],
    num_rows: 9586
})

## Tokenizer basically goes through the dataset and 
1. Adds start and end pad
2. Trims the convs that exceeds the max_length
3. Seperates each word/token in the conversation


In [15]:
en_nlp = spacy.load("en_core_web_sm")

In [17]:
def tokenize_example(example, en_nlp, max_length, lower, sos_token, eos_token):
    en_tokens_1 = [token.text for token in en_nlp.tokenizer(example["Key"])][:max_length]
    en_tokens_2 = [token.text for token in en_nlp.tokenizer(example["Data"])][:max_length]
    if lower:
        en_tokens_1 = [token.lower() for token in en_tokens_1]
        en_tokens_2 = [token.lower() for token in en_tokens_2]
    en_tokens_1 = [sos_token] + en_tokens_1 + [eos_token]
    en_tokens_2 = [sos_token] + en_tokens_2 + [eos_token]
    return {"conv1": en_tokens_1, "conv2": en_tokens_2}

# def tokenize_example(example, en_nlp, max_length, lower, sos_token, eos_token):
#     en_tokens_1 = [token.text for token in en_nlp.tokenizer(example["Patient"])][:max_length]
#     en_tokens_2 = [token.text for token in en_nlp.tokenizer(example["Doctor"])][:max_length]
#     if lower:
#         en_tokens_1 = [token.lower() for token in en_tokens_1]
#         en_tokens_2 = [token.lower() for token in en_tokens_2]
#     en_tokens_1 = [sos_token] + en_tokens_1 + [eos_token]
#     en_tokens_2 = [sos_token] + en_tokens_2 + [eos_token]
#     return {"conv1": en_tokens_1, "conv2": en_tokens_2}

In [18]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map: 100%|█████████████████████████████████████████████████████████| 1348/1348 [00:00<00:00, 20562.63 examples/s]


In [23]:
train_data[2]

{'Key': 'U',
 'Data': 'What answers did everyone put?',
 'conv1': ['<sos>', 'u', '<eos>'],
 'conv2': ['<sos>', 'what', 'answers', 'did', 'everyone', 'put', '?', '<eos>']}

## Next we need to create a Vocabulary to associate each token with a number, since our model will not get the words as an input but the index of the word from the Vocabulary

In [30]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab_1 = torchtext.vocab.build_vocab_from_iterator(
    train_data["conv1"],
    min_freq=min_freq,
    specials=special_tokens,
)

# en_vocab_2 = torchtext.vocab.build_vocab_from_iterator(
#     train_data["conv2"],
#     min_freq=min_freq,
#     specials=special_tokens,
# )

In [32]:
# en_vocab.get_itos()
# en_vocab.get_stoi()
# dict(sorted(en_vocab_1.get_stoi().items(), key=lambda item: item[1]))
# en_vocab_1.lookup_token(4)
# en_vocab.lookup_indices(['a'])

In [31]:
en_vocab_1.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 'the',
 ',',
 'i',
 'a',
 'and',
 'to',
 '.',
 'it',
 'is',
 'we',
 'that',
 '?',
 'rule',
 'have',
 'vowel',
 'even']

In [ ]:
# len(en_vocab_1), len(en_vocab_2)

In [33]:
# assert en_vocab_1[unk_token] == en_vocab_2[unk_token]
# assert en_vocab_1[pad_token] == en_vocab_2[pad_token]

unk_index = en_vocab_1[unk_token]
pad_index = en_vocab_1[pad_token]

In [34]:
en_vocab_1.set_default_index(unk_index)
# en_vocab_2.set_default_index(unk_index)

In [40]:
tokens = ["i", "think", "rule", "choice", "shows", "movie"]

In [41]:
en_vocab_1.lookup_indices(tokens)

[6, 23, 16, 324, 611, 0]

In [42]:
en_vocab_1.lookup_tokens(en_vocab_1.lookup_indices(tokens))

['i', 'think', 'rule', 'choice', 'shows', '<unk>']

## Convert tokens to indices

In [43]:
def numericalize_example(example, en_vocab_1, en_vocab_2=None):
    en_ids = en_vocab_1.lookup_indices(example["conv1"])
    # de_ids = en_vocab_2.lookup_indices(example["conv2"])
    de_ids = en_vocab_1.lookup_indices(example["conv2"])
    return {"conv1_ids": en_ids, "conv2_ids": de_ids}

In [44]:
# fn_kwargs = {"en_vocab_1": en_vocab_1, "en_vocab_2": en_vocab_2}
fn_kwargs = {"en_vocab_1": en_vocab_1}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map: 100%|█████████████████████████████████████████████████████████| 1348/1348 [00:00<00:00, 30519.01 examples/s]


In [45]:
train_data[0]

{'Key': 'Zebra,Beaver,Duck,Chipmunk',
 'Data': 'B,U,7,2',
 'conv1': ['<sos>',
  'zebra',
  ',',
  'beaver',
  ',',
  'duck',
  ',',
  'chipmunk',
  '<eos>'],
 'conv2': ['<sos>', 'b', ',', 'u,7,2', '<eos>'],
 'conv1_ids': [2, 239, 5, 209, 5, 189, 5, 216, 3],
 'conv2_ids': [2, 261, 5, 1241, 3]}

In [46]:
en_vocab_1.lookup_tokens(train_data[0]["conv1_ids"])

['<sos>', 'zebra', ',', 'beaver', ',', 'duck', ',', 'chipmunk', '<eos>']

## Since model expects number in tensor format we convert them

In [47]:
data_type = "torch"
format_columns = ["conv1_ids", "conv2_ids"]

train_data = train_data.with_format(
    type=data_type, 
    columns=format_columns, 
    output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [48]:
train_data[0]

{'conv1_ids': tensor([  2, 239,   5, 209,   5, 189,   5, 216,   3]),
 'conv2_ids': tensor([   2,  261,    5, 1241,    3]),
 'Key': 'Zebra,Beaver,Duck,Chipmunk',
 'Data': 'B,U,7,2',
 'conv1': ['<sos>',
  'zebra',
  ',',
  'beaver',
  ',',
  'duck',
  ',',
  'chipmunk',
  '<eos>'],
 'conv2': ['<sos>', 'b', ',', 'u,7,2', '<eos>']}

## Since, training is done in batches we create batch here

In [55]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids_1 = [example["conv1_ids"] for example in batch]
        batch_en_ids_2 = [example["conv2_ids"] for example in batch]
        batch_en_ids_1 = nn.utils.rnn.pad_sequence(batch_en_ids_1, padding_value=pad_index)
        batch_en_ids_2 = nn.utils.rnn.pad_sequence(batch_en_ids_2, padding_value=pad_index)
        batch = {
            "conv1_ids": batch_en_ids_1,
            "conv2_ids": batch_en_ids_2,
        }
        return batch

    return collate_fn

In [56]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [57]:
batch_size = 8
## generally 8,16,32,64 are used to create batch size

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## Building the LSTM RNN model

In [58]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [59]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [61]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [62]:
input_dim = len(en_vocab_1)
output_dim = len(en_vocab_1)
encoder_embedding_dim = 128
decoder_embedding_dim = 128
hidden_dim = 256
n_layers = 3
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [63]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(1272, 128)
    (rnn): LSTM(128, 256, num_layers=3, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1272, 128)
    (rnn): LSTM(128, 256, num_layers=3, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=1272, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [64]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 3,548,408 trainable parameters


In [65]:
optimizer = optim.Adam(model.parameters())

In [66]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [67]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["conv1_ids"].to(device)
        trg = batch["conv2_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [68]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["conv1_ids"].to(device)
            trg = batch["conv2_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [69]:
torch.cuda.empty_cache()

In [70]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        test_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|███████▋                                                                     | 1/10 [00:13<01:57, 13.04s/it]

	Train Loss:   4.788 | Train PPL: 120.066
	Valid Loss:   4.770 | Valid PPL: 117.926


 20%|███████████████▍                                                             | 2/10 [00:25<01:42, 12.87s/it]

	Train Loss:   4.466 | Train PPL:  86.986
	Valid Loss:   4.782 | Valid PPL: 119.387


 30%|███████████████████████                                                      | 3/10 [00:37<01:27, 12.54s/it]

...

	Train Loss:   4.049 | Train PPL:  57.358
	Valid Loss:   5.047 | Valid PPL: 155.505


 90%|█████████████████████████████████████████████████████████████████████▎       | 9/10 [01:56<00:13, 13.02s/it]

	Train Loss:   4.019 | Train PPL:  55.634
	Valid Loss:   5.005 | Valid PPL: 149.231


100%|████████████████████████████████████████████████████████████████████████████| 10/10 [02:08<00:00, 12.90s/it]

	Train Loss:   4.004 | Train PPL:  54.796
	Valid Loss:   4.782 | Valid PPL: 119.345


In [71]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 4.770 | Test PPL: 117.926 |


## Test the prediction capability

In [72]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    en_vocab_1,
    en_vocab_2,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in en_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = en_vocab_1.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        # print(tensor)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab_2.lookup_indices([sos_token])
        # print(inputs)
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab_2[eos_token]:
                break
        tokens = en_vocab_2.lookup_tokens(inputs)
        # print(tokens)
    return tokens

In [75]:
sentence = train_data[5]["Key"]
expected_translation = train_data[5]["Data"]

sentence, expected_translation

('I put 7 and U', 'I put U and 2')

In [76]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    en_vocab_1,
    en_vocab_1,
    lower,
    sos_token,
    eos_token,
    device,
)

In [77]:
translation

['<sos>', 'i', '<eos>']

### We can now loop over our `test_data`, getting our model's translation of each test sentence.


In [80]:
translations = [
    translate_sentence(
        example["Key"],
        model,
        en_nlp,
        en_vocab_1,
        en_vocab_1,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_data)
]


100%|██████████████████████████████████████████████████████████████████████| 1348/1348 [00:00<00:00, 1485.88it/s]


# Quality Assurance

## To check our models ability we use a metric called BLEU


In [87]:
bleu = evaluate.load("bleu")


  0%|                                                                                   | 0/1348 [03:22<?, ?it/s]


In [81]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["Data"]] for example in test_data]

In [82]:
predictions[0], references[0]

('i', ['3,L,A,4'])

In [83]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [84]:
tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

In [85]:
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

(['i'], ['3,l', ',', 'a,4'])

Finally, we calculate the BLEU metric across our test set!

We pass our `predictions`, `references` and our `tokenizer_fn` to the `compute` method of the BLEU metric to get our results.


In [88]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [89]:
results

{'bleu': 0.0,
 'precisions': [0.22774480712166173, 0.0, 0.0, 0.0],
 'brevity_penalty': 0.000301250996410356,
 'length_ratio': 0.10979881078439357,
 'translation_length': 1348,
 'reference_length': 12277}